In [ ]:
%pip install clip-retrieval img2dataset pandas numpy 

In [2]:
%pip install torchvision>=0.10.1,<2 --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117Note: you may need to restart the kernel to use updated packages.



In [1]:
import torch
import os
import pandas as pd
import numpy as np
from clip_retrieval import clip_inference
import shutil
import torch
from torch.utils.data import TensorDataset, DataLoader

print(torch.cuda.is_available())

base_folder = os.path.join("F:", "nude_sexy_safe_v1_x320", "testing")
output_base_folder = os.path.join("F:", "nsfw_embeddings")
categories = ["nude", "sexy", "safe"]

c:\Users\coold\Documents\GitHub\clip-nsfw-detector\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [3]:
for i, c in enumerate(categories):
    folder = os.path.join(base_folder, c)
    output_folder = os.path.join(output_base_folder, c)
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    # https://github.com/rom1504/clip-retrieval/issues/220
    clip_inference(folder, output_folder, num_prepro_workers=0, clip_model="ViT-L/14", batch_size=32)

The number of samples has been estimated to be 3878
Starting the worker
dataset is 37
Starting work on task 0
warming up with batch size 32 on cuda
done warming up in 3.021339178085327s
The number of samples has been estimated to be 2121
Starting the worker
dataset is 37
Starting work on task 0
The number of samples has been estimated to be 4050
Starting the worker
dataset is 37
Starting work on task 0


In [2]:
cat_embeddings = {}
for c in categories:
    cat_embeddings[c] = np.load(os.path.join(output_base_folder, c, "img_emb", "img_emb_0.npy"))

pos_x = np.concatenate([cat_embeddings["nude"], cat_embeddings["sexy"]])
neg_x = cat_embeddings["safe"]

num_pos = pos_x.shape[0]
num_neg = neg_x.shape[0]

pos_y = np.ones((num_pos, 1))
neg_y = np.zeros((num_neg, 1))

x_np = np.vstack([pos_x, neg_x])
y_np = np.vstack([pos_y, neg_y])

In [3]:
x = torch.from_numpy(x_np).type(torch.float16)
y = torch.from_numpy(y_np).type(torch.float16)
print(x.dtype, y.dtype)
train_dataset = TensorDataset(x, y)

# Define the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

torch.float16 torch.float16


In [4]:
baseline = torch.from_numpy(np.load(r"F:\nsfw_testset\nsfw_testset\drawings-test\img_emb\img_emb_0.npy"))
print(baseline.shape)

torch.Size([853, 768])


In [6]:
import torch
import torch.nn as nn
from torch.optim import Adam, AdamW
from tqdm import tqdm
import wandb

class H14_NSFW_Detector(nn.Module):
    def __init__(self, input_size=1024):
        super().__init__()
        self.input_size = input_size
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            nn.GELU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 16),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.layers(x)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = H14_NSFW_Detector(768).to(device)
criterion = nn.MSELoss().to(device)
optimizer = AdamW(model.parameters())
num_epochs = 100

epoch = 0

wandb.init(project="nsfw-detefctor")
wandb.watch(model)

with torch.cuda.amp.autocast():
    # Training loop
    for epoch in range(num_epochs):
        train_iter = tqdm(train_loader, total=len(train_loader))
        for inputs, labels in train_iter:
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Clear gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)
            train_iter.set_postfix(loss=loss.item(), epoch=epoch)
            wandb.log({"loss": loss.item(), "epoch": epoch, "lr": optimizer.param_groups[0]['lr']})

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

loss,▇▆▄▂▃█▃▆▇▇▃▄▂▅▅▆▆▇▆▅▇▅▇▇▆▃█▆▇▅▅▆▄█▆▅▅▄▁▆
loss,0.02549


100%|██████████| 315/315 [00:01<00:00, 161.79it/s, epoch=9, loss=0.536]
